In [1]:
import pandas as pd

Nhập các file vào để kiểm tra

In [33]:
GR = pd.read_csv('Ontop_General_Records.csv')
BF = pd.read_csv('Ontop_Customer_Business_Fields.csv')
CD = pd.read_csv('Ontop_Customer_Details.csv')
OD = pd.read_csv('Ontop_Order_Details.csv')
RTE = pd.read_csv('Ontop_Records_Types_Explanation.csv')
S = pd.read_csv('Ontop_Salary.csv')
SD = pd.read_csv('Ontop_Staff_Details.csv')
SUD = pd.read_csv('Ontop_Supplier_Details.csv')
SG = pd.read_csv('Ontop_Supplier_Geolocation.csv')

Xem info từng file

In [4]:
SG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Supplier_Area_ID     10 non-null     object 
 1   Industrial_Name      10 non-null     object 
 2   Industrial_province  10 non-null     object 
 3   geolocation_lat      10 non-null     float64
 4   geolocation_lng      10 non-null     float64
 5   Supplier_Area        10 non-null     object 
dtypes: float64(2), object(4)
memory usage: 612.0+ bytes


Dataframe GR và OD có null, xem nó là gì và tiến hành xử lý
Trước tiên là với file GR

In [5]:
GR[GR['Payee_ID'].isnull()]
# => Cột này có 1 giá trị null ở Payee_ID, dò cột PAYEE= ZOOBIENGE để xem có giá trị nào khác tương tự không

,DATE,Record_Type,PAYEE,Payee_ID,Internal Code,Description,Amount
6011,10/31/2022,COS,ZOOBIENGE,NaN,ZOOB2216,"PAID 50% Deposit for Furniture, ZOOB2216-2","66,050.85"


In [6]:
GR[GR['PAYEE'] == "ZOOBIENGE"]
# => Nhận thấy giá trị ZOOBIENGE là duy nhất trong bảng GR.
# => Do trong Description có ghi là Paid => Là 1 supplier => dò trong bảng 

,DATE,Record_Type,PAYEE,Payee_ID,Internal Code,Description,Amount
6011,10/31/2022,COS,ZOOBIENGE,NaN,ZOOB2216,"PAID 50% Deposit for Furniture, ZOOB2216-2","66,050.85"


In [7]:
Replace_ID = SUD[SUD['Supplier_Name'] == "ZOOBIENGE"]["Supplier_ID"]
# => Thấy dc Supplier_ID là SP253

In [8]:
Replace_ID

252    SP253
Name: Supplier_ID, dtype: object

In [ ]:
# Thay giá trị null bằng SP253
GR['Payee_ID'] = GR['Payee_ID'].fillna(value = Replace_ID)

In [10]:
GR['Payee_ID'] = GR['Payee_ID'].fillna(value = 'SP253')

In [11]:
# Xuất lại file sau khi làm sạch
GR.to_csv('Ontop_General_Records_cleaned.csv')

Tiếp theo là với file OD

In [34]:
OD[OD['Customer_ID'].isnull()]
# => Giá trị null là Customer_ID ứng với khách mã khách là ZOOB (chữ cái đầu tiên của đơn hàng)

,Date,Customer_ID,Internal_Code,Project_Revenue,Project_Cost,Project_Profit
505,2022-11-05,NaN,ZOOB2207,13415.79,14868.91,-1453.12


In [35]:
# Thay thế chỗ Null bằng ZOOB
OD['Customer_ID'] = OD['Customer_ID'].fillna(value = 'ZOOB')

In [ ]:
OD[OD['Customer_ID'] == "ZOOB"]['Internal_Code'].value_counts()
# => Thấy rằng ZOOB2207 bị chia thành 02 hàng, nên sẽ tiến hành cộng lại

In [49]:
# Tính lại tổng các cột
Sums = OD.groupby('Internal_Code').agg({'Project_Revenue':'sum','Project_Cost':'sum','Project_Profit':'sum'}).reset_index()

In [50]:
# Xóa các cột cũ và merger 2 bảng lại với nhau
OD.drop(columns=['Project_Revenue' , 'Project_Cost' , 'Project_Profit'], inplace=True)

In [52]:
OD = OD.merge(Sums , how= 'inner', on= 'Internal_Code')

In [54]:
# Xuất file sau khi làm sạch
OD.to_csv('Ontop_Order_Details_cleaned.csv')